# Clustering Toronto Neighborhoods

#### Import libraries

In [156]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Import neighborhood data and visualize on the map

In [157]:
df = pd.read_csv('TorontoNeighborhoodData.csv')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [158]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


##### Select a subset of neighborhoods with name containing 'Toronto'

In [159]:
data = df[df['Borough'].str.contains ('Toronto')].reset_index(drop=True)
data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [160]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data['Borough'].unique()),
        data.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


##### Get Toronto coordinates

In [161]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


##### Create a map of Toronto neighborhoods

In [162]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

#### Use Foursquare API to get information on neighborhood venues

In [163]:
CLIENT_ID = 'XPWW0VGBU2F5XPERQWY1RG5LMB4HGQFAPSZZK5DXZ3DASFTZ' # your Foursquare ID
CLIENT_SECRET = 'TES1JPBUX2UHIR1ZZWG3QMHCAOCOGDSFHKRXO5S5POBQVTAR' # your Foursquare Secret
VERSION = '20181101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XPWW0VGBU2F5XPERQWY1RG5LMB4HGQFAPSZZK5DXZ3DASFTZ
CLIENT_SECRET:TES1JPBUX2UHIR1ZZWG3QMHCAOCOGDSFHKRXO5S5POBQVTAR


##### Define function to extract nearby venues for all neighborhoods

In [164]:
def getNearbyVenues(names, latitudes, longitudes, radius=700, LIMIT = 150):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Run above function on each neighborhood and create dataframe toronto_venues

In [165]:
toronto_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

##### The size of the resulting dataframe:

In [166]:
print(toronto_venues.shape)
toronto_venues.head()

(2467, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
2,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
3,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub


##### Number of venues for each neighborhood

In [167]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",87,87,87,87,87,87
Business reply mail Processing Centre969 Eastern,43,43,43,43,43,43
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",21,21,21,21,21,21
"Cabbagetown, St. James Town",75,75,75,75,75,75
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,24,24,24,24,24,24


##### Remove neighborhoods where the number of venues is less than 10

In [168]:
toronto_venues = toronto_venues.groupby('Neighborhood').filter(lambda x: len(x)>10)
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",87,87,87,87,87,87
Business reply mail Processing Centre969 Eastern,43,43,43,43,43,43
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",21,21,21,21,21,21
"Cabbagetown, St. James Town",75,75,75,75,75,75
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,24,24,24,24,24,24


In [169]:
print('The dataframe has {} neighborhoods.'.format(
        toronto_venues.groupby('Neighborhood').count().shape[0]
    )
)

The dataframe has 33 neighborhoods.


##### Number of unique venue categories

In [170]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


#### Analyze Neighborhoods

In [171]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

col_num = toronto_onehot.columns.get_loc('Neighborhood')

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[col_num]]+ list(toronto_onehot.columns[0:col_num]) 
fixed_columns = fixed_columns + list(toronto_onehot.columns[col_num+1:-1]) 

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [172]:
toronto_onehot.shape

(2439, 262)

##### Group rows by venues frequency

In [173]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.00,0.010000,0.01,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.00,0.00,0.00,0.0

In [174]:
toronto_grouped.shape

(33, 262)

In [175]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2        Hotel  0.05
3   Steakhouse  0.04
4    Gastropub  0.03


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1        Hotel  0.06
2   Restaurant  0.05
3         Café  0.05
4          Pub  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.07
1                    Café  0.05
2                     Bar  0.03
3  Furniture / Home Store  0.02
4           Burrito Place  0.02


----Business reply mail Processing Centre969 Eastern----
                  venue  freq
0  Fast Food Restaurant  0.09
1    Light Rail Station  0.05
2                  Park  0.05
3           Coffee Shop  0.05
4       Harbor / Marina  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0  Airport Terminal  0.14
1     Boat or Ferry  0.10
2    Airport

##### A function that sorts the venues in descending order.

In [176]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Create a dataframe with most frequent venues by neighborhood¶

In [177]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Steakhouse,American Restaurant,Sushi Restaurant,Theater,Gastropub,Bar,Restaurant
1,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Cocktail Bar,Pub,Deli / Bodega,Hotel Bar,Sporting Goods Shop,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Bar,Furniture / Home Store,Thrift / Vintage Store,Breakfast Spot,Restaurant,Burrito Place,Performing Arts Venue,BBQ Joint
3,Business reply mail Processing Centre969 Eastern,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Harbor / Marina,Park,Burrito Place,Light Rail Station,Restaurant,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Harbor / Marina,Sculpture Garden,Boat or Ferry,Airport Lounge,Airport Service,Boutique,Music Venue,Plane,Coffee Shop
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Park,Japanese Restaurant,Diner,Bakery,Gastropub,Indian Restaurant
6,Central Bay Street,Coffee Shop,Café,Art Gallery,Arts & Crafts Store,Italian Restaurant,Bubble Tea Shop,Clothing Store,Chinese Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Chinese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Dessert Shop,Vietnamese Restaurant
8,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Italian Restaurant,Diner,Nightclub,Convenience Store,Playground
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Burger Joint,Gay Bar,Café,Diner,Restaurant,Bubble Tea Shop,Breakfast Spot,Ramen Restaurant


### Cluster Neighborhoods

In [178]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 1, 3, 2, 1, 1, 2, 1], dtype=int32)

##### Create a new dataframe that includes cluster and top 10 venues for each neighborhood

In [181]:
toronto_merged = pd.merge(data, neighborhoods_venues_sorted, on='Neighborhood', how='inner')

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Bar,Pub,Pet Store,Breakfast Spot,Sandwich Place,Coffee Shop,Café,Gastropub,Vegetarian / Vegan Restaurant,Pizza Place,4
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Greek Restaurant,Coffee Shop,Pub,Ice Cream Shop,Restaurant,Grocery Store,Fast Food Restaurant,Café,Spa,Sandwich Place,4
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,Indian Restaurant,Café,Pizza Place,Fast Food Restaurant,Pet Store,Coffee Shop,Park,Sandwich Place,Gym,Butcher,1
3,M4M,East Toronto,Studio District,43.659526,-79.340923,Bakery,Coffee Shop,Café,Bar,Diner,American Restaurant,Sandwich Place,Italian Restaurant,Juice Bar,Latin American Restaurant,1
4,M4P,Central Toronto,Davisville North,43.712751,-79.390197,Gym,Park,Ice Cream Shop,Taco Place,Café,Food & Drink Shop,Indian Restaurant,Deli / Bodega,Dessert Shop,Thai Restaurant,3


In [182]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

##### Cluster 1

In [183]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
12,St. James Town,Café,Coffee Shop,Restaurant,Hotel,Bakery,Italian Restaurant,Breakfast Spot,Cosmetics Shop,American Restaurant,Seafood Restaurant,0
31,"Runnymede, Swansea",Café,Coffee Shop,Pizza Place,Italian Restaurant,Gastropub,Falafel Restaurant,Sushi Restaurant,Pub,Restaurant,Bakery,0


##### Cluster 2

In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,"The Beaches West, India Bazaar",Indian Restaurant,Café,Pizza Place,Fast Food Restaurant,Pet Store,Coffee Shop,Park,Sandwich Place,Gym,Butcher,1
3,Studio District,Bakery,Coffee Shop,Café,Bar,Diner,American Restaurant,Sandwich Place,Italian Restaurant,Juice Bar,Latin American Restaurant,1
6,Davisville,Coffee Shop,Pizza Place,Café,Dessert Shop,Sandwich Place,Restaurant,Fast Food Restaurant,Pharmacy,Italian Restaurant,Seafood Restaurant,1
7,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Italian Restaurant,Sandwich Place,Sushi Restaurant,Café,Pub,Skating Rink,Pizza Place,Gym,Pharmacy,1
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Burger Joint,Gay Bar,Café,Diner,Restaurant,Bubble Tea Shop,Breakfast Spot,Ramen Restaurant,1
13,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Cocktail Bar,Pub,Deli / Bodega,Hotel Bar,Sporting Goods Shop,Steakhouse,1
15,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Steakhouse,American Restaurant,Sushi Restaurant,Theater,Gastropub,Bar,Restaurant,1
17,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Concert Hall,Sports Bar,Steakhouse,Gastropub,Restaurant,Italian Restaurant,1
19,"The Annex, North Midtown, Yorkville",Coffee Shop,Pizza Place,Sandwich Place,Café,Vegetarian / Vegan Restaurant,American Restaurant,Burger Joint,Pharmacy,Indian Restaurant,Pub,1
20,"Harbord, University of Toronto",Coffee Shop,Café,Bar,Bookstore,Pizza Place,Restaurant,Pub,Thai Restaurant,Sandwich Place,Japanese Restaurant,1


##### Cluster 3

In [185]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
5,North Toronto West,Coffee Shop,Sporting Goods Shop,Clothing Store,Café,Italian Restaurant,Diner,Restaurant,Hotel,Bakery,Steakhouse,2
8,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Park,Japanese Restaurant,Diner,Bakery,Gastropub,Indian Restaurant,2
11,"Ryerson, Garden District",Coffee Shop,Clothing Store,Restaurant,Café,Tea Room,Diner,Bar,Department Store,Falafel Restaurant,Hotel,2
22,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Harbor / Marina,Sculpture Garden,Boat or Ferry,Airport Lounge,Airport Service,Boutique,Music Venue,Plane,Coffee Shop,2
23,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Cocktail Bar,Gastropub,Bakery,Italian Restaurant,Pub,2
24,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Deli / Bodega,Gastropub,American Restaurant,Bakery,Tea Room,2


##### Cluster 4

In [186]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
4,Davisville North,Gym,Park,Ice Cream Shop,Taco Place,Café,Food & Drink Shop,Indian Restaurant,Deli / Bodega,Dessert Shop,Thai Restaurant,3


##### Cluster 5

In [187]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,The Beaches,Bar,Pub,Pet Store,Breakfast Spot,Sandwich Place,Coffee Shop,Café,Gastropub,Vegetarian / Vegan Restaurant,Pizza Place,4
1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Pub,Ice Cream Shop,Restaurant,Grocery Store,Fast Food Restaurant,Café,Spa,Sandwich Place,4
10,"Harbourfront, Regent Park",Coffee Shop,Café,Park,Bakery,Theater,Restaurant,Pub,Performing Arts Venue,Breakfast Spot,Gym / Fitness Center,4
14,Central Bay Street,Coffee Shop,Café,Art Gallery,Arts & Crafts Store,Italian Restaurant,Bubble Tea Shop,Clothing Store,Chinese Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,4
16,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Italian Restaurant,Park,Sports Bar,Scenic Lookout,Brewery,Aquarium,Pizza Place,Café,4
18,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Steakhouse,Japanese Restaurant,Seafood Restaurant,4
26,"Dovercourt Village, Dufferin",Coffee Shop,Gym,Park,Supermarket,Pharmacy,Bakery,Middle Eastern Restaurant,Bar,Italian Restaurant,Café,4
27,"Little Portugal, Trinity",Bar,Restaurant,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Cocktail Bar,Wine Bar,Pizza Place,Ice Cream Shop,Asian Restaurant,4
